In [10]:
import pickle
import pandas as pd
import numpy as np
import math
with open("./conveni_total_df.pkl","rb") as f:
    df= pickle.load(f)

In [11]:
df.head(1)

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,CU,가평군청점,경기도,0315817570,NaN,NaN,NaN,NaN,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",coffee,True


In [12]:
df.shape

(640498, 11)

In [13]:
df.values.ndim #차원을이야기해준다 

2

In [14]:
pip install pymysql #mysql패키지에 직접붙는 패키지

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pymysql

In [16]:
con =pymysql.connect(host='ip', user='id', password='password', db='encore', charset='utf8')
cur = con.cursor() #private ip는 내부에서만 유효하다 명시적으로 3306인데 vim mysql파일들어가서 port고칠수있다
#error뜰시 sudo service mysql start

In [17]:
cur.execute("show tables") #서브넷마스크 172.23.78.91/24
# 앞에 24비트를 연산하면 그대로 host ip 뒤에 8개는 각각 우리 #브로드캐스트 
# 8개만허용 254개를 묶을수있다 앞에 24개가 바뀌면 라우터가 바껴야한다
# 다른 곳이랑 바꿀러며 라우터가필요 뒤에서브넷은 공간을 자른거

3

In [18]:
cur

In [19]:
df.head(1)

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,CU,가평군청점,경기도,0315817570,NaN,NaN,NaN,NaN,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",coffee,True


In [20]:
for x in df.columns:
    print(f"'{x}'")


'brand'
'store_name'
'sido'
'store_tel'
'store_code'
'lat'
'longs'
'open_time'
'store_address'
'service'
'available'


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 640498 entries, 0 to 186119
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   brand          640498 non-null  object
 1   store_name     640498 non-null  object
 2   sido           640498 non-null  object
 3   store_tel      193963 non-null  object
 4   store_code     260415 non-null  object
 5   lat            446475 non-null  object
 6   longs          446475 non-null  object
 7   open_time      186120 non-null  object
 8   store_address  640498 non-null  object
 9   service        640498 non-null  object
 10  available      640498 non-null  bool  
dtypes: bool(1), object(10)
memory usage: 54.4+ MB


In [22]:
# 결측치 데이텨형태 nan float형
df[df.store_address.notnull()].store_address.apply(lambda x : len(x)).max()

89

In [23]:
# CREATE Table StorageEvent
# (
#     `brand` VARCHAR(10) NOT NULL,
#     `store_name` VARCHAR(30) NOT NULL,
#     `sido`  CHAR(7) NOT NULL,
#     `store_tel`  CHAR(14)  Default NULL,
#     `store_code` CHAR(5) Default NULL,
#     `lat` FLOAT Default null,
#     `longs` FLOAT Default null,
#     `open_time` CHAR(20) Default null,
#     `store_address` VARCHAR(100) NOT NULL,
#     `service` VARCHAR(20)  NOT NULL,
#     `available` INT NOT
#     PRIMARY KEY(`store_name`,`store_address`)
# )

# mysql에서도 확인해보기
# heidisql에서도 만들어보기
# 블록처리는 컨트롤 f9
# 전체실행은 f9
# sql alchemy dajngeo orm이 아닌다

In [24]:
# 하드코딩
# sql = "INSERT INTO store VALUES ('엔코아', '서초동이야', '서울', '02-777-7777', 
# 							NULL, NULL, NULL, '08:00~21:30', '효령로 ....', '공부', 1)"

sql = "INSERT INTO store VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s)"

In [25]:
pd.notnull(df) #결측치인지 아닌지확인해준다 nan-결측치 공백-입력값

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,True,True,True,True,False,False,False,False,True,True,True
1,True,True,True,True,False,False,False,False,True,True,True
2,True,True,True,True,False,False,False,False,True,True,True
3,True,True,True,True,False,False,False,False,True,True,True
4,True,True,True,True,False,False,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
186115,True,True,True,False,False,True,True,True,True,True,True
186116,True,True,True,False,False,True,True,True,True,True,True
186117,True,True,True,False,False,True,True,True,True,True,True
186118,True,True,True,False,False,True,True,True,True,True,True


In [26]:
df.store_tel = df.store_tel.replace("",None) #None은 객체가없는 파이썬에서의 None, NaN은 numpy의 misisngvalue 데이터가아니란거

In [27]:
?df.where #false라고 되어있는걸 바꾸어준다

Object `df.where #false라고 되어있는걸 바꾸어준다` not found.


In [28]:
df2 = df.where(pd.notnull(df),None) #NaN->None으로 대

In [29]:
pd.notnull(df).head(2)

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,True,True,True,True,False,False,False,False,True,True,True
1,True,True,True,False,False,False,False,False,True,True,True


In [30]:
df.where(pd.notnull(df))

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,CU,가평군청점,경기도,0315817570,NaN,NaN,NaN,NaN,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",coffee,True
1,CU,가평레일파크점,경기도,NaN,NaN,NaN,NaN,NaN,경기도 가평군 가평읍 연인길 2 (읍내리),coffee,True
2,CU,가평본점,경기도,NaN,NaN,NaN,NaN,NaN,경기도 가평군 가화로 110-1 (가평읍) 1층,coffee,True
3,CU,가평센타점,경기도,0315817470,NaN,NaN,NaN,NaN,경기도 가평군 가평읍 오리나무길 9 1층,coffee,True
4,CU,가평오렌지점,경기도,NaN,NaN,NaN,NaN,NaN,"경기도 가평군 가평읍 가화로 174, (읍내리)",coffee,True
...,...,...,...,...,...,...,...,...,...,...,...
186115,seven,충주터미널점,충청북도,NaN,NaN,36.9816000752929,127.914367824229,,충청북도 충주시 봉계1길 49,토토,False
186116,seven,충주푸르지오점,충청북도,NaN,NaN,36.9774964700015,127.913212488916,,"충청북도 충주시 봉계8길 2 , 1층",토토,False
186117,seven,충주호암점,충청북도,NaN,NaN,36.955341151,127.94029493,,충청북도 충주시 호암중앙2로 32 101호(호암동),토토,False
186118,seven,충주호암LH점,충청북도,NaN,NaN,36.9554585474234,127.927072494247,,충청북도 충주시 호암수청2로 56,토토,False


In [31]:
df2

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",coffee,True
1,CU,가평레일파크점,경기도,None,None,None,None,None,경기도 가평군 가평읍 연인길 2 (읍내리),coffee,True
2,CU,가평본점,경기도,None,None,None,None,None,경기도 가평군 가화로 110-1 (가평읍) 1층,coffee,True
3,CU,가평센타점,경기도,0315817470,None,None,None,None,경기도 가평군 가평읍 오리나무길 9 1층,coffee,True
4,CU,가평오렌지점,경기도,None,None,None,None,None,"경기도 가평군 가평읍 가화로 174, (읍내리)",coffee,True
...,...,...,...,...,...,...,...,...,...,...,...
186115,seven,충주터미널점,충청북도,None,None,36.9816000752929,127.914367824229,,충청북도 충주시 봉계1길 49,토토,False
186116,seven,충주푸르지오점,충청북도,None,None,36.9774964700015,127.913212488916,,"충청북도 충주시 봉계8길 2 , 1층",토토,False
186117,seven,충주호암점,충청북도,None,None,36.955341151,127.94029493,,충청북도 충주시 호암중앙2로 32 101호(호암동),토토,False
186118,seven,충주호암LH점,충청북도,None,None,36.9554585474234,127.927072494247,,충청북도 충주시 호암수청2로 56,토토,False


In [32]:
df2.iloc[0].values.tolist() #array values를 리스트형태로 전달

['CU',
 '가평군청점',
 '경기도',
 '0315817570',
 None,
 None,
 None,
 None,
 '경기도 가평군 가평읍 석봉로 175, (읍내리) 1호',
 'coffee',
 True]

In [33]:
df2.available = df2.available.astype(int)

In [34]:
# sql alchemy 안쓰기 
cur.execute(sql, df2.iloc[0].values.tolist()) #데이터안들어감 commit해야함

1

In [35]:
con.commit()

In [36]:
"%s %d" % ("hi",10)

'hi 10'

In [37]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [38]:
from tqdm import tqdm

In [39]:
for idx, row in tqdm(df2.iterrows()):
    if idx==0:
        continue;
    try:
       cur.execute(sql, row.values.tolist()) 
    except Exception as e:
        print(e)
    
    # break #pk값때문에 중복되어서 오류뜬다

640498it [02:52, 3711.39it/s]


In [40]:
con.commit()

In [72]:
row.tolist()

['CU',
 '가평군청점',
 '경기도',
 '0315817570',
 None,
 None,
 None,
 None,
 '경기도 가평군 가평읍 석봉로 175, (읍내리) 1호',
 'coffee',
 1]

In [43]:
df2[df2.store_name=="가평군청점"]

,brand,store_name,sido,store_tel,store_code,lat,longs,open_time,store_address,service,available
0,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",coffee,1
17633,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",delivery,1
35266,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",toto,0
52899,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",fried,1
70532,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",lotto,0
88165,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",cash,1
105798,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",pos,0
123431,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",bakery,0
141064,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",mix,0
158697,CU,가평군청점,경기도,0315817570,None,None,None,None,"경기도 가평군 가평읍 석봉로 175, (읍내리) 1호",battery,0
